# Exercícios de revisão

1. A empresa decidiu criar um programa de recompensas que premia o vendedor que teve melhor performânce em cada mês. Para isso, pediu ao time de dados um relatório que traga o nome e o valor total vendido (incluído descontos) do melhor vendedor de cada mês.

In [ ]:
-- A empresa decidiu criar um programa de recompensas que premia o vendedor que teve melhor performânce em cada mês. 
--Para isso, pediu ao time de dados um relatório que traga o nome e o valor total vendido (incluído descontos) do melhor vendedor de cada mês.

WITH tabela_temporaria AS
(SELECT 
	TO_CHAR(orders.order_date, 'YYYY-MM') AS mes,
	orders.employee_id,
	SUM(order_details.unit_price * order_details.quantity * (1 - order_details.discount)) AS venda_total,
	RANK () OVER (PARTITION BY TO_CHAR(orders.order_date, 'YYYY-MM') ORDER BY  SUM(order_details.unit_price * order_details.quantity * (1 - order_details.discount)) DESC) AS ranque
FROM orders
LEFT JOIN order_details ON orders.order_id = order_details.order_id
GROUP BY 
	TO_CHAR(orders.order_date, 'YYYY-MM'),
	orders.employee_id)

SELECT
	mes,
	tabela_temporaria.employee_id,
	CONCAT(employees.first_name, ' ', employees.last_name) AS nome_completo,
	ROUND(venda_total::numeric, 2) AS vendas_totais
FROM tabela_temporaria
LEFT JOIN employees ON tabela_temporaria.employee_id = employees.employee_id
WHERE ranque = 1

2. Quais vendedores ganharam o prêmio de vendedor do mês mais de uma vez e quantas vezes eles ganharam?

In [ ]:
-- Quais vendedores ganharam o prêmio de vendedor do mês mais de uma vez e quantas vezes eles ganharam?

CREATE OR REPLACE VIEW vendedores_do_mes AS
(WITH tabela_temporaria AS
(SELECT 
	TO_CHAR(orders.order_date, 'YYYY-MM') AS mes,
	orders.employee_id,
	SUM(order_details.unit_price * order_details.quantity * (1 - order_details.discount)) AS venda_total,
	RANK () OVER (PARTITION BY TO_CHAR(orders.order_date, 'YYYY-MM') ORDER BY  SUM(order_details.unit_price * order_details.quantity * (1 - order_details.discount)) DESC) AS ranque
FROM orders
LEFT JOIN order_details ON orders.order_id = order_details.order_id
GROUP BY 
	TO_CHAR(orders.order_date, 'YYYY-MM'),
	orders.employee_id)

SELECT
	mes,
	tabela_temporaria.employee_id,
	CONCAT(employees.first_name, ' ', employees.last_name) AS nome_completo,
	ROUND(venda_total::numeric, 2) AS vendas_totais
FROM tabela_temporaria
LEFT JOIN employees ON tabela_temporaria.employee_id = employees.employee_id
WHERE ranque = 1);

SELECT 
	employee_id,
	nome_completo,
	COUNT(employee_id) AS quantidade_empregado_do_mes
FROM vendedores_do_mes
GROUP BY employee_id, nome_completo
HAVING COUNT(employee_id) > 2